In [147]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 163 kB in 2s (73.2 kB/s)
Reading package lists... Don

In [148]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-14 01:28:21--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.14’

postgresql-42.2.16. 100%[===================>] 979.38K  5.72MB/s    in 0.2s    

2022-05-14 01:28:22 (5.72 MB/s) - ‘postgresql-42.2.16.jar.14’ saved [1002883/1002883]



In [149]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("breastcancer").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [150]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://breastcancer-mackalys.s3.amazonaws.com/data_fixed.csv"
spark.sparkContext.addFile(url)
breast_cancer_df = spark.read.csv(SparkFiles.get("data_fixed.csv"), sep=",", header=True, inferSchema=True)

In [151]:
# Print our DataFrame
breast_cancer_df.show()

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave_points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave_points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave_points_worst|symmetry_worst|fractal_dimension_worst|
+--------+---------+-----------+------------+---

In [152]:
# Show the columns
breast_cancer_df.columns

['id',
 'diagnosis',
 'radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave_points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave_points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'radius_worst',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave_points_worst',
 'symmetry_worst',
 'fractal_dimension_worst']

In [153]:
# Describe our data
breast_cancer_df.describe()

DataFrame[summary: string, id: string, diagnosis: string, radius_mean: string, texture_mean: string, perimeter_mean: string, area_mean: string, smoothness_mean: string, compactness_mean: string, concavity_mean: string, concave_points_mean: string, symmetry_mean: string, fractal_dimension_mean: string, radius_se: string, texture_se: string, perimeter_se: string, area_se: string, smoothness_se: string, compactness_se: string, concavity_se: string, concave_points_se: string, symmetry_se: string, fractal_dimension_se: string, radius_worst: string, texture_worst: string, perimeter_worst: string, area_worst: string, smoothness_worst: string, compactness_worst: string, concavity_worst: string, concave_points_worst: string, symmetry_worst: string, fractal_dimension_worst: string]

In [155]:
# Define the DataFrames
data_csv_df = breast_cancer_df.select(["id", "diagnosis","radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean", "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"])
data_csv_df.show()

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave_points_mean|symmetry_mean|fractal_dimension_mean|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave_points_worst|symmetry_worst|fractal_dimension_worst|
+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+--

In [156]:
# Define the DataFrames
area_mean_radius_mean_df = breast_cancer_df.select(["id", "diagnosis","radius_mean", "area_mean"])
area_mean_radius_mean_df.show()

+--------+---------+-----------+---------+
|      id|diagnosis|radius_mean|area_mean|
+--------+---------+-----------+---------+
|  842302|        M|      17.99|   1001.0|
|  842517|        M|      20.57|   1326.0|
|84300903|        M|      19.69|   1203.0|
|84348301|        M|      11.42|    386.1|
|84358402|        M|      20.29|   1297.0|
|  843786|        M|      12.45|    477.1|
|  844359|        M|      18.25|   1040.0|
|84458202|        M|      13.71|    577.9|
|  844981|        M|       13.0|    519.8|
|84501001|        M|      12.46|    475.9|
|  845636|        M|      16.02|    797.8|
|84610002|        M|      15.78|    781.0|
|  846226|        M|      19.17|   1123.0|
|  846381|        M|      15.85|    782.7|
|84667401|        M|      13.73|    578.3|
|84799002|        M|      14.54|    658.8|
|  848406|        M|      14.68|    684.5|
|84862001|        M|      16.13|    798.8|
|  849014|        M|      19.81|   1260.0|
| 8510426|        B|      13.54|    566.3|
+--------+-

In [157]:
# Define the DataFrames
radius_worst_perimeter_mean_df = breast_cancer_df.select(["id", "diagnosis", "perimeter_mean", "radius_worst"])
radius_worst_perimeter_mean_df.show()

+--------+---------+--------------+------------+
|      id|diagnosis|perimeter_mean|radius_worst|
+--------+---------+--------------+------------+
|  842302|        M|         122.8|       25.38|
|  842517|        M|         132.9|       24.99|
|84300903|        M|         130.0|       23.57|
|84348301|        M|         77.58|       14.91|
|84358402|        M|         135.1|       22.54|
|  843786|        M|         82.57|       15.47|
|  844359|        M|         119.6|       22.88|
|84458202|        M|          90.2|       17.06|
|  844981|        M|          87.5|       15.49|
|84501001|        M|         83.97|       15.09|
|  845636|        M|         102.7|       19.19|
|84610002|        M|         103.6|       20.42|
|  846226|        M|         132.4|       20.96|
|  846381|        M|         103.7|       16.84|
|84667401|        M|          93.6|       15.03|
|84799002|        M|         96.73|       17.46|
|  848406|        M|         94.74|       19.07|
|84862001|        M|

In [158]:
# Define the DataFrames
texture_worst_texture_mean_df = breast_cancer_df.select(["id", "diagnosis", "texture_mean", "texture_worst"])
texture_worst_texture_mean_df.show()

+--------+---------+------------+-------------+
|      id|diagnosis|texture_mean|texture_worst|
+--------+---------+------------+-------------+
|  842302|        M|       10.38|        17.33|
|  842517|        M|       17.77|        23.41|
|84300903|        M|       21.25|        25.53|
|84348301|        M|       20.38|         26.5|
|84358402|        M|       14.34|        16.67|
|  843786|        M|        15.7|        23.75|
|  844359|        M|       19.98|        27.66|
|84458202|        M|       20.83|        28.14|
|  844981|        M|       21.82|        30.73|
|84501001|        M|       24.04|        40.68|
|  845636|        M|       23.24|        33.88|
|84610002|        M|       17.89|        27.28|
|  846226|        M|        24.8|        29.94|
|  846381|        M|       23.95|        27.66|
|84667401|        M|       22.61|        32.01|
|84799002|        M|       27.54|        37.13|
|  848406|        M|       20.13|        30.88|
|84862001|        M|       20.68|       

In [159]:
# Define the DataFrames
radius_mean_area_worst_df = breast_cancer_df.select(["id", "diagnosis","radius_mean", "area_worst"])
radius_mean_area_worst_df.show()

+--------+---------+-----------+----------+
|      id|diagnosis|radius_mean|area_worst|
+--------+---------+-----------+----------+
|  842302|        M|      17.99|    2019.0|
|  842517|        M|      20.57|    1956.0|
|84300903|        M|      19.69|    1709.0|
|84348301|        M|      11.42|     567.7|
|84358402|        M|      20.29|    1575.0|
|  843786|        M|      12.45|     741.6|
|  844359|        M|      18.25|    1606.0|
|84458202|        M|      13.71|     897.0|
|  844981|        M|       13.0|     739.3|
|84501001|        M|      12.46|     711.4|
|  845636|        M|      16.02|    1150.0|
|84610002|        M|      15.78|    1299.0|
|  846226|        M|      19.17|    1332.0|
|  846381|        M|      15.85|     876.5|
|84667401|        M|      13.73|     697.7|
|84799002|        M|      14.54|     943.2|
|  848406|        M|      14.68|    1138.0|
|84862001|        M|      16.13|    1315.0|
|  849014|        M|      19.81|    2398.0|
| 8510426|        B|      13.54|

In [160]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://breast-cancer-db.cqavoh8kz2dp.us-east-1.rds.amazonaws.com:5432/breast-cancer-db"
config = {"user":"postgres",
          "password": "SCNewport2",
          "driver":"org.postgresql.Driver"}

In [161]:
# Write data_csv_df to table in RDS
data_csv_df.write.jdbc(url=jdbc_url, table='data_csv', mode=mode, properties=config)
# Write area_mean_radius_mean_df to table in RDS
area_mean_radius_mean_df.write.jdbc(url=jdbc_url, table='area_mean_radius_mean', mode=mode, properties=config)
# Write radius_worst_perimeter_mean_df to table in RDS
radius_worst_perimeter_mean_df.write.jdbc(url=jdbc_url, table='radius_worst_perimeter_mean', mode=mode, properties=config)
# Write texture_worst_texture_mean_df to table in RDS
texture_worst_texture_mean_df.write.jdbc(url=jdbc_url, table='texture_worst_texture_mean', mode=mode, properties=config)
# Write radius_mean_area_worst_df to table in RDS
radius_mean_area_worst_df.write.jdbc(url=jdbc_url, table='radius_mean_area_worst_df', mode=mode, properties=config)